In [1]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.declarative import declared_attr

from sqlalchemy.orm import sessionmaker

In [2]:
engine = create_engine("sqlite:///:memory:", echo=True)

In [3]:
from sqlalchemy import Column, Integer, String, Boolean
from sqlalchemy.orm import relationship
from sqlalchemy import ForeignKey

Base = declarative_base()
Session = sessionmaker(bind=engine)
session = Session()

Base.query = Session.query()

AttributeError: 'sessionmaker' object has no attribute 'query'

In [4]:
class BaseTable(object):
    
    """
    Base class for NCBI taxonomy SQLalchemy tables.
    Intended to be subclassed, will not work on it's own.
    
    Defines several class methods for working with the SQLAlchemy ORM.
    """

    @declared_attr
    def __tablename__(cls):
        return cls.__name__.lower()
    
    # Always have dedicated primary key
    id = Column(Integer, primary_key=True)

    @classmethod
    def from_file(cls, filepath, session, sep=None, columns=None, header=None):
        """ Read a file into the SQL taxonomy database. """
        if columns is None:
            columns = cls.columns
        
        if sep is None:
            sep = cls.sep
        
        if header is None:
            header = cls.header
        
        def line_trans(line):
            output = {}
            sline = line.split(sep)
            for column, (colname, trans) in zip(sline, columns):
                output[colname] = trans(column) 
            return output

        with open(filepath, "r") as handle:
            #if there is a header skip the first line. 
            if header:
                next(handle)

            session.bulk_insert_mappings(
                cls,
                [line_trans(l) for l in handle]
            )

        session.commit()
        return


class Acc2Tax(BaseTable, Base):
    """ Table to hold any of the files mapping accession to taxid.
    
    Columns in the table are:
    accession
    accession.version
    taxid
    gi
    
    Note that these files may have a header column
    """
    
    accession = Column(String)
    accession_version = Column(String, unique=True)
    
    taxid = Column(Integer, ForeignKey('nodes.taxid'))
    node = relationship("Nodes", back_populates="accessions")
    
    gi = Column(Integer)

    # Columns class variable is used for reading from txt files.
    # Can be changed when invoking the class method
    columns = [
        ("accession", str),
        ("accession_version", str),
        ("taxid", int),
        ("gi", int),
        ]

    # The column delimiter for the text file.
    sep = "\t"
    header = True


class Nodes(BaseTable, Base):
    """
    Table schema to hold "nodes.dmp" from taxdump folder.

    Description of file columns in NCBI ftp:
    tax_id -- node id in GenBank taxonomy database
    parent tax_id -- parent node id in GenBank taxonomy database
    rank -- rank of this node (superkingdom, kingdom, ...) 
    embl code -- locus-name prefix; not unique
    division id -- see division.dmp file
    inherited div flag  (1 or 0) -- 1 if node inherits division from parent
    genetic code id -- see gencode.dmp file
    inherited GC  flag  (1 or 0) -- 1 if node inherits genetic code from parent
    mitochondrial genetic code id -- see gencode.dmp file
    inherited MGC flag  (1 or 0) -- 1 if node inherits mitochondrial gencode from parent
    GenBank hidden flag (1 or 0) -- 1 if name is suppressed in GenBank entry lineage
    hidden subtree root flag (1 or 0) -- 1 if this subtree has no sequence data yet
    comments -- free-text comments and citations
    """

    taxid = Column(Integer, unique=True)
    parent_taxid = Column(Integer)
    rank = Column(String)
    embl_code = Column(String)
    
    division_id = Column(Integer, ForeignKey("division.division_id")) # Foreign key
    division = relationship("Division", back_populates="nodes")

    inherited_div_flag = Column(Boolean)
    genetic_code_id = Column(String) # Foreign key
    inherited_genetic_code_flag = Column(Boolean)
    mitochonchondrial_genetic_code_id = Column(String) # Foreign key, poss same as genetic code id.
    inherited_mitochonchondrial_genetic_code_flag = Column(Boolean)
    genbank_hidden_flag = Column(Boolean)
    hidden_subtree_root_flag = Column(Boolean)
    comment = Column(String)

    names = relationship("Names", back_populates="node")
    accessions = relationship("Acc2Tax", back_populates="node")

    # Columns class variable is used for reading from txt files.
    # Can be changed when invoking the class method
    columns = [
        ("taxid", int),
        ("parent_taxid", int),
        ("rank", str),
        ("embl_code", str),
        ("division_id", int),
        ("inherited_div_flag", bool),
        ("genetic_code_id", str),
        ("inherited_genetic_code_flag", bool),
        ("mitochonchondrial_genetic_code_id", str),
        ("inherited_mitochonchondrial_genetic_code_flag", bool),
        ("genbank_hidden_flag", bool),
        ("hidden_subtree_root_flag", bool), 
        ("comments", str)
        ]

    # The file delimiter for the text file.
    sep = "\t|\t"
    header = False


    @classmethod
    def get_parents(cls, taxids, session):
        def recurse(taxid):
            output = []
            
            record = session.query(cls).filter(cls.taxid == taxid).one()
            output.append(record)
            
            if taxid != 1:
                output.extend(recurse(record.parent_taxid))
            
            return output
        
        output = []
        for taxid in taxids:
            output.extend(recurse(taxid))
        return output

    def get_children(cls, taxids, session=None):
        
        if session is not None and cls.session is not None:
            session = cls.session

class Names(BaseTable, Base):
    """ Table schema for "names.dmp" from taxdmp folder

    Description of file columns:
    tax_id -- the id of node associated with this name
    name_txt -- name itself
    unique name -- the unique variant of this name if name not unique
    name class -- (synonym, common name, ...)
    """

    taxid = Column(Integer, ForeignKey('nodes.taxid'))
    name = Column(String)
    unique_name = Column(String)
    name_class = Column(String)

    node = relationship("Nodes", back_populates="names")

    # Columns class variable is used for reading from txt files.
    # Can be changed when invoking the class method
    columns = [
        ("taxid", int),
        ("name", str),
        ("unique_name", str),
        ("name_class", str),
        ]

    # The file delimiter for the text file.
    sep = "\t|\t"
    header = False


class Division(BaseTable, Base):
    """ Table schema for "divisions.dmp" from taxdmp folder

    Description of file columns:
    division id -- taxonomy database division id
    division cde -- GenBank division code (three characters)
    division name -- e.g. BCT, PLN, VRT, MAM, PRI...
    comments
    """

    division_id = Column(Integer, unique=True) # Foreign key with nodes.
    division_cde = Column(String) # Three letter name
    division_name = Column(String)
    comments = Column(String)

    nodes = relationship("Nodes", back_populates="division")
    
    # Columns class variable is used for reading from txt files.
    # Can be changed when invoking the class method
    columns = [
        ("division_id", int),
        ("division_cde", str),
        ("division_name", str),
        ("comments", str),
        ]

    # The file delimiter for the text file.
    sep = "\t|\t"
    header = False


class GenCode(BaseTable, Base):
    """ Table schema for "gencode.dmp" from taxdmp folder.

    ##Has a weird file format, needs special parser

    Description of file columns:

    genetic code id -- GenBank genetic code id
    abbreviation -- genetic code name abbreviation
    name -- genetic code name
    cde -- translation table for this genetic code
    starts -- start codons for this genetic code

    """

    code_id = Column(String)
    abbreviation = Column(String)
    name = Column(String)
    cde = Column(String)
    starts = Column(String)

In [5]:
Base.metadata.create_all(engine)

2018-01-31 15:39:57,371 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-01-31 15:39:57,372 INFO sqlalchemy.engine.base.Engine ()
2018-01-31 15:39:57,373 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-01-31 15:39:57,373 INFO sqlalchemy.engine.base.Engine ()
2018-01-31 15:39:57,374 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("acc2tax")
2018-01-31 15:39:57,375 INFO sqlalchemy.engine.base.Engine ()
2018-01-31 15:39:57,377 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("nodes")
2018-01-31 15:39:57,377 INFO sqlalchemy.engine.base.Engine ()
2018-01-31 15:39:57,378 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("names")
2018-01-31 15:39:57,379 INFO sqlalchemy.engine.base.Engine ()
2018-01-31 15:39:57,380 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("division")
2018-01-31 15:39:57,381 INFO sqlalchemy.engine.base.Engine ()
2018-01-31 15:39:57,382 INFO sqlalchemy.e

In [6]:
Nodes.from_file("/home/darcyabjones/data/sequence_databases/data/ncbi_taxonomy/nodes.dmp", session)

2018-01-31 15:40:03,091 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-01-31 15:40:29,080 INFO sqlalchemy.engine.base.Engine INSERT INTO nodes (taxid, parent_taxid, rank, embl_code, division_id, inherited_div_flag, genetic_code_id, inherited_genetic_code_flag, mitochonchondrial_genetic_code_id, inherited_mitochonchondrial_genetic_code_flag, genbank_hidden_flag, hidden_subtree_root_flag) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2018-01-31 15:40:29,081 INFO sqlalchemy.engine.base.Engine ((1, 1, 'no rank', '', 8, 1, '1', 1, '0', 1, 1, 1), (2, 131567, 'superkingdom', '', 0, 1, '11', 1, '0', 1, 1, 1), (6, 335928, 'genus', '', 0, 1, '11', 1, '0', 1, 1, 1), (7, 6, 'species', 'AC', 0, 1, '11', 1, '0', 1, 1, 1), (9, 32199, 'species', 'BA', 0, 1, '11', 1, '0', 1, 1, 1), (10, 1706371, 'genus', '', 0, 1, '11', 1, '0', 1, 1, 1), (11, 1707, 'species', 'CG', 0, 1, '11', 1, '0', 1, 1, 1), (13, 203488, 'genus', '', 0, 1, '11', 1, '0', 1, 1, 1)  ... displaying 10 of 1671956 total bound para

In [7]:
Names.from_file("/home/darcyabjones/data/sequence_databases/data/ncbi_taxonomy/names.dmp", session)

2018-01-31 15:40:37,182 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-01-31 15:40:53,009 INFO sqlalchemy.engine.base.Engine INSERT INTO names (taxid, name, unique_name, name_class) VALUES (?, ?, ?, ?)
2018-01-31 15:40:53,010 INFO sqlalchemy.engine.base.Engine ((1, 'all', '', 'synonym\t|\n'), (1, 'root', '', 'scientific name\t|\n'), (2, 'Bacteria', 'Bacteria <prokaryotes>', 'scientific name\t|\n'), (2, 'Monera', 'Monera <Bacteria>', 'in-part\t|\n'), (2, 'Procaryotae', 'Procaryotae <Bacteria>', 'in-part\t|\n'), (2, 'Prokaryota', 'Prokaryota <Bacteria>', 'in-part\t|\n'), (2, 'Prokaryotae', 'Prokaryotae <Bacteria>', 'in-part\t|\n'), (2, 'bacteria', 'bacteria <blast2>', 'blast name\t|\n')  ... displaying 10 of 2502749 total bound parameter sets ...  (2070753, 'Sagenomella sclerotialis', '', 'synonym\t|\n'), (2070766, 'Psoralea aphylla complex', '', 'scientific name\t|\n'))
2018-01-31 15:40:56,518 INFO sqlalchemy.engine.base.Engine COMMIT


In [14]:
x = Nodes.get_parents([2070753, 2070766], session)
x

2018-01-31 15:47:06,959 INFO sqlalchemy.engine.base.Engine SELECT nodes.id AS nodes_id, nodes.taxid AS nodes_taxid, nodes.parent_taxid AS nodes_parent_taxid, nodes.rank AS nodes_rank, nodes.embl_code AS nodes_embl_code, nodes.division_id AS nodes_division_id, nodes.inherited_div_flag AS nodes_inherited_div_flag, nodes.genetic_code_id AS nodes_genetic_code_id, nodes.inherited_genetic_code_flag AS nodes_inherited_genetic_code_flag, nodes.mitochonchondrial_genetic_code_id AS nodes_mitochonchondrial_genetic_code_id, nodes.inherited_mitochonchondrial_genetic_code_flag AS nodes_inherited_mitochonchondrial_genetic_code_flag, nodes.genbank_hidden_flag AS nodes_genbank_hidden_flag, nodes.hidden_subtree_root_flag AS nodes_hidden_subtree_root_flag, nodes.comment AS nodes_comment 
FROM nodes 
WHERE nodes.taxid = ?
2018-01-31 15:47:06,960 INFO sqlalchemy.engine.base.Engine (2070753,)
2018-01-31 15:47:06,964 INFO sqlalchemy.engine.base.Engine SELECT nodes.id AS nodes_id, nodes.taxid AS nodes_taxid, 

2018-01-31 15:47:06,994 INFO sqlalchemy.engine.base.Engine (4890,)
2018-01-31 15:47:06,997 INFO sqlalchemy.engine.base.Engine SELECT nodes.id AS nodes_id, nodes.taxid AS nodes_taxid, nodes.parent_taxid AS nodes_parent_taxid, nodes.rank AS nodes_rank, nodes.embl_code AS nodes_embl_code, nodes.division_id AS nodes_division_id, nodes.inherited_div_flag AS nodes_inherited_div_flag, nodes.genetic_code_id AS nodes_genetic_code_id, nodes.inherited_genetic_code_flag AS nodes_inherited_genetic_code_flag, nodes.mitochonchondrial_genetic_code_id AS nodes_mitochonchondrial_genetic_code_id, nodes.inherited_mitochonchondrial_genetic_code_flag AS nodes_inherited_mitochonchondrial_genetic_code_flag, nodes.genbank_hidden_flag AS nodes_genbank_hidden_flag, nodes.hidden_subtree_root_flag AS nodes_hidden_subtree_root_flag, nodes.comment AS nodes_comment 
FROM nodes 
WHERE nodes.taxid = ?
2018-01-31 15:47:06,999 INFO sqlalchemy.engine.base.Engine (451864,)
2018-01-31 15:47:07,001 INFO sqlalchemy.engine.bas

2018-01-31 15:47:07,029 INFO sqlalchemy.engine.base.Engine (3814,)
2018-01-31 15:47:07,032 INFO sqlalchemy.engine.base.Engine SELECT nodes.id AS nodes_id, nodes.taxid AS nodes_taxid, nodes.parent_taxid AS nodes_parent_taxid, nodes.rank AS nodes_rank, nodes.embl_code AS nodes_embl_code, nodes.division_id AS nodes_division_id, nodes.inherited_div_flag AS nodes_inherited_div_flag, nodes.genetic_code_id AS nodes_genetic_code_id, nodes.inherited_genetic_code_flag AS nodes_inherited_genetic_code_flag, nodes.mitochonchondrial_genetic_code_id AS nodes_mitochonchondrial_genetic_code_id, nodes.inherited_mitochonchondrial_genetic_code_flag AS nodes_inherited_mitochonchondrial_genetic_code_flag, nodes.genbank_hidden_flag AS nodes_genbank_hidden_flag, nodes.hidden_subtree_root_flag AS nodes_hidden_subtree_root_flag, nodes.comment AS nodes_comment 
FROM nodes 
WHERE nodes.taxid = ?
2018-01-31 15:47:07,032 INFO sqlalchemy.engine.base.Engine (3803,)
2018-01-31 15:47:07,035 INFO sqlalchemy.engine.base.

2018-01-31 15:47:07,069 INFO sqlalchemy.engine.base.Engine (58024,)
2018-01-31 15:47:07,070 INFO sqlalchemy.engine.base.Engine SELECT nodes.id AS nodes_id, nodes.taxid AS nodes_taxid, nodes.parent_taxid AS nodes_parent_taxid, nodes.rank AS nodes_rank, nodes.embl_code AS nodes_embl_code, nodes.division_id AS nodes_division_id, nodes.inherited_div_flag AS nodes_inherited_div_flag, nodes.genetic_code_id AS nodes_genetic_code_id, nodes.inherited_genetic_code_flag AS nodes_inherited_genetic_code_flag, nodes.mitochonchondrial_genetic_code_id AS nodes_mitochonchondrial_genetic_code_id, nodes.inherited_mitochonchondrial_genetic_code_flag AS nodes_inherited_mitochonchondrial_genetic_code_flag, nodes.genbank_hidden_flag AS nodes_genbank_hidden_flag, nodes.hidden_subtree_root_flag AS nodes_hidden_subtree_root_flag, nodes.comment AS nodes_comment 
FROM nodes 
WHERE nodes.taxid = ?
2018-01-31 15:47:07,071 INFO sqlalchemy.engine.base.Engine (78536,)
2018-01-31 15:47:07,073 INFO sqlalchemy.engine.bas

In [15]:
[[y_i.name for y_i in x_i.names] for x_i in x]

2018-01-31 15:47:14,975 INFO sqlalchemy.engine.base.Engine SELECT names.id AS names_id, names.taxid AS names_taxid, names.name AS names_name, names.unique_name AS names_unique_name, names.name_class AS names_name_class 
FROM names 
WHERE ? = names.taxid
2018-01-31 15:47:14,978 INFO sqlalchemy.engine.base.Engine (2070766,)
2018-01-31 15:47:15,146 INFO sqlalchemy.engine.base.Engine SELECT names.id AS names_id, names.taxid AS names_taxid, names.name AS names_name, names.unique_name AS names_unique_name, names.name_class AS names_name_class 
FROM names 
WHERE ? = names.taxid
2018-01-31 15:47:15,146 INFO sqlalchemy.engine.base.Engine (100167,)
2018-01-31 15:47:15,305 INFO sqlalchemy.engine.base.Engine SELECT names.id AS names_id, names.taxid AS names_taxid, names.name AS names_name, names.unique_name AS names_unique_name, names.name_class AS names_name_class 
FROM names 
WHERE ? = names.taxid
2018-01-31 15:47:15,306 INFO sqlalchemy.engine.base.Engine (163737,)
2018-01-31 15:47:15,464 INFO s

[['Aspergillus sclerotialis',
  'Aspergillus sclerotialis (W. Gams & Breton) Houbraken et al., 2017',
  'CBS 366.77',
  'IAM 14794',
  'Phialosimplex sclerotialis',
  'Sagenomella sclerotialis'],
 ['Aspergillus', 'Petromyces'],
 ['Aspergillaceae', 'Aspergillaceae Link 1826'],
 ['Elaphomycetales', 'Eurotiales', 'green and blue molds'],
 ['Eurotiomycetidae'],
 ['Chaetothyriomycetes',
  'Eurotiomycetes',
  'Loculoascomycetes',
  'Plectomycetes',
  'bitunicate ascomycetes'],
 ['leotiomyceta'],
 ['Euascomycota', 'Pezizomycotina'],
 ['saccharomyceta'],
 ['Ascomycota', 'ascomycetes', 'ascomycetes', 'sac fungi'],
 ['Dikarya'],
 ['Fungi', 'fungi', 'fungi'],
 ['Fungi/Metazoa group',
  'Opisthokonta',
  'Opisthokonta Cavalier-Smith 1987',
  'opisthokonts'],
 ['Eucarya',
  'Eucaryotae',
  'Eukarya',
  'Eukaryota',
  'Eukaryotae',
  'eucaryotes',
  'eukaryotes',
  'eukaryotes'],
 ['biota', 'cellular organisms'],
 ['all', 'root'],
 ['Psoralea aphylla complex'],
 ['Psoralea', 'Psoralea L., 1753'],
 [

In [11]:
y = x[0].division

2018-01-30 18:07:22,352 INFO sqlalchemy.engine.base.Engine SELECT division.id AS division_id_1, division.division_id AS division_division_id, division.division_cde AS division_division_cde, division.division_name AS division_division_name, division.comments AS division_comments 
FROM division 
WHERE division.division_id = ?
2018-01-30 18:07:22,356 INFO sqlalchemy.engine.base.Engine ('plants',)


In [15]:
y.division_name

'PLN'

In [16]:
with open("/home/darcyabjones/data/sequence_databases/data/ncbi_taxonomy/nodes.dmp") as handle:
    for i, f in enumerate(handle):
        print(f)
        if i > 5:
            break

1	|	1	|	no rank	|		|	8	|	0	|	1	|	0	|	0	|	0	|	0	|	0	|		|

2	|	131567	|	superkingdom	|		|	0	|	0	|	11	|	0	|	0	|	0	|	0	|	0	|		|

6	|	335928	|	genus	|		|	0	|	1	|	11	|	1	|	0	|	1	|	0	|	0	|		|

7	|	6	|	species	|	AC	|	0	|	1	|	11	|	1	|	0	|	1	|	1	|	0	|		|

9	|	32199	|	species	|	BA	|	0	|	1	|	11	|	1	|	0	|	1	|	1	|	0	|		|

10	|	1706371	|	genus	|		|	0	|	1	|	11	|	1	|	0	|	1	|	0	|	0	|		|

11	|	1707	|	species	|	CG	|	0	|	1	|	11	|	1	|	0	|	1	|	1	|	0	|		|

